Python version of the matlab file `compare_weights_spatially_controlForSorting.m`

In [3]:
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy
import pandas as pd
import scipy.stats as stats
import scipy.sparse as sparse
import copy
import time
import warnings 
from scipy import integrate
import seaborn as sns
from numpy import sqrt, cosh, tanh, pi, exp
warnings.simplefilter('ignore')

import scipy.io as sio
import os
from tqdm.auto import tqdm

# using functions from PyalData
from pyaldata import *

The control for spike sorting is as follows:
- 1) run this for each neuron
- 2) copy some percentage of spikes to all neighboring cells
- 3) recalculate PCA
- 4) compare the chosen neuron to all others
- 5) move on to the next neuron and repeat

In [4]:
percent_spikes = [1,2,3,5,10]
num_iters = 10
which_dim = 1
binz = np.arange(0, 0.005, 0.5) # start, step, stop

## 1. Load data and convert to pandas df

In [10]:
'''
Insert path and filename 
'''

path = '/Users/Daphne/Data/'  
fname = 'Chewie_CO_VR_2016-10-06.mat' 
mat = scipy.io.loadmat(path + fname) 

In [11]:
'''
Convert matlab file to df
'''
df = mat2dataframe(path+fname, shift_idx_fields=True)

In [12]:
'''
Session parameters
'''

sessions = [...] # TODO: for all sessions (currently only doing one session)

task = 'CO'

# set the start and end indices 
num_bins = 3
which_period = 'all' 

if which_period == 'plan':
    idx_start = df['idx_target_on'][0]
    idx_end = df['idx_go_cue'][0]
    
elif which_period == 'move':
    idx_start = df['idx_go_cue'][0]
    idx_end = df['idx_trial_end'][0]

elif which_period == 'all':
    idx_start = df['idx_target_on'][0]
    idx_end = df['idx_trial_end'][0]

In [ ]:
eigen_sum = np.zeros([len(sessions), 3]) #TODO: where does the 3 come from??

In [ ]:
#for s in sessions:

file = 